In [16]:
import pandas as pd
import numpy as np
import sys
import os

# Allows to get the module in utils
sys.path.append(os.path.abspath(".."))


from utils.load import load_data
from utils.preprocessing import preprocess
from utils.split import split_data

from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
# from sklearn.compose import ColumnTransformer

import tensorflow as tf

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import class_weight

In [17]:
# Load data
df = load_data()

In [18]:
# Split and preprocess datas set
X_train, X_val, X_test, y_train, y_val, y_test = split_data(df)

X_train_processed, X_test_processed, X_val_processed, y_test_encoding, y_train_encoded, y_val_encoding, pipeline, le = preprocess(X_train, X_val, X_test, y_train, y_val, y_test)

/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [19]:
X_train_dense = X_train_processed.toarray() if hasattr(X_train_processed, "toarray") else X_train_processed
X_val_dense = X_val_processed.toarray() if hasattr(X_val_processed, "toarray") else X_val_processed
X_test_dense = X_test_processed.toarray() if hasattr(X_test_processed, "toarray") else X_test_processed

In [20]:
y_train_vector = y_train_encoded.reshape(-1)
y_val_vector = y_val_encoding.reshape(-1)
y_test_vector = y_test_encoding.reshape(-1)

In [21]:
input_dim = X_train_dense.shape[1]

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(input_dim,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss='binary_crossentropy',
    metrics=['accuracy', 'recall', 'auc']
)

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │       290,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 294,593 (1.12 MB)

 Trainable params: 294,593 (1.12 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',    # surveille la perte de validation
    patience=3,            # tolère 3 époques sans amélioration
    restore_best_weights=True
)

In [23]:
history = model.fit(
    X_train_dense, y_train_vector,
    validation_data=(X_val_dense, y_val_vector),
    epochs=20,
    batch_size=16,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7720 - auc: 0.8080 - loss: 0.4621 - recall: 0.4971 - val_accuracy: 0.8146 - val_auc: 0.8456 - val_loss: 0.4226 - val_recall: 0.5084
Epoch 2/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8559 - auc: 0.9037 - loss: 0.3415 - recall: 0.6482 - val_accuracy: 0.7595 - val_auc: 0.8106 - val_loss: 0.5093 - val_recall: 0.6823
Epoch 3/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9860 - auc: 0.9978 - loss: 0.0506 - recall: 0.9629 - val_accuracy: 0.6495 - val_auc: 0.6891 - val_loss: 1.1851 - val_recall: 0.7157
Epoch 4/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9920 - auc: 0.9994 - loss: 0.0222 - recall: 0.9830 - val_accuracy: 0.7879 - val_auc: 0.8119 - val_loss: 0.5043 - val_recall: 0.4114


In [24]:
y_pred_probs = model.predict(X_test_dense)
y_pred = np.argmax(y_pred_probs, axis=1)
print("\nClassification Report :")
print(classification_report(y_test_vector, y_pred))

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Classification Report :
              precision    recall  f1-score   support

           0       0.73      1.00      0.85      1035
           1       0.00      0.00      0.00       374

    accuracy                           0.73      1409
   macro avg       0.37      0.50      0.42      1409
weighted avg       0.54      0.73      0.62      1409



/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

In [25]:
print("Matrice de Confusion :")
cm = confusion_matrix(y_test_vector, y_pred)
print(cm)

Matrice de Confusion :
[[1035    0]
 [ 374    0]]


In [26]:
y_pred_probs = model.predict(X_test_dense)

probs = y_pred_probs.ravel()

y_pred = (probs >= 0.5)
print("\nClassification Report :")
print(classification_report(y_test_vector, y_pred))
y_test_vector.shape, y_pred_probs.shape

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 992us/step

Classification Report :
              precision    recall  f1-score   support

           0       0.83      0.91      0.87      1035
           1       0.65      0.47      0.55       374

    accuracy                           0.79      1409
   macro avg       0.74      0.69      0.71      1409
weighted avg       0.78      0.79      0.78      1409



((1409,), (1409, 1))